[View in Colaboratory](https://colab.research.google.com/github/ami432/INKERS/blob/master/Copy_of_Bharatkumar_Batch3_Assignment4B_img_aug_imp.ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size =64
num_classes = 10
epochs = 40
l = 12
num_filter = 28
compression = 0.5
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 41s 0us/step


In [0]:
# In Keras, An augmented image generator can be created as follows:	
# set up image augmentation
datagen = ImageDataGenerator( 
							zca_whitening=False,						
							rotation_range = 10, 
							width_shift_range=0.1,
							height_shift_range=0.1, 							
							fill_mode='nearest',						
							horizontal_flip = True )

# Fit image generator on the Data
datagen.fit(x_train)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 28
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [11]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 28)   756         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 28)   112         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 28)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
# checkpoint
filepath="CIFAR10-imgaug-weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='auto',period=1)
callbacks_list = [checkpoint]


In [14]:
# Fit the mode
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=epochs,callbacks=callbacks_list,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/40
782/782 [==============================] - 389s 498ms/step - loss: 1.5645 - acc: 0.4206 - val_loss: 1.6839 - val_acc: 0.4875

Epoch 00001: saving model to CIFAR10-imgaug-weights-improvement-01-0.49.hdf5
Epoch 2/40
 90/782 [==>...........................] - ETA: 5:16 - loss: 1.2657 - acc: 0.5462

782/782 [==============================] - 376s 481ms/step - loss: 1.1660 - acc: 0.5779 - val_loss: 1.4700 - val_acc: 0.5625

Epoch 00002: saving model to CIFAR10-imgaug-weights-improvement-02-0.56.hdf5
Epoch 3/40
147/782 [====>.........................] - ETA: 4:49 - loss: 1.0599 - acc: 0.6197

782/782 [==============================] - 376s 481ms/step - loss: 0.9896 - acc: 0.6462 - val_loss: 3.0489 - val_acc: 0.4227

Epoch 00003: saving model to CIFAR10-imgaug-weights-improvement-03-0.42.hdf5
Epoch 4/40
163/782 [=====>........................] - ETA: 4:42 - loss: 0.9138 - acc: 0.6777

782/782 [==============================] - 375s 480ms/step - loss: 0.8914 - acc: 0.6835 - val_loss: 1.8354 - val_acc: 0.5758

Epoch 00004: saving model to CIFAR10-imgaug-weights-improvement-04-0.58.hdf5
Epoch 5/40
167/782 [=====>........................] - ETA: 4:42 - loss: 0.8408 - acc: 0.7010

782/782 [==============================] - 376s 481ms/step - loss: 0.8240 - acc: 0.7083 - val_loss: 1.6189 - val_acc: 0.6051

Epoch 00005: saving model to CIFAR10-imgaug-weights-improvement-05-0.61.hdf5
Epoch 6/40
168/782 [=====>........................] - ETA: 4:40 - loss: 0.7806 - acc: 0.7227

782/782 [==============================] - 375s 480ms/step - loss: 0.7625 - acc: 0.7322 - val_loss: 1.0482 - val_acc: 0.7007

Epoch 00006: saving model to CIFAR10-imgaug-weights-improvement-06-0.70.hdf5
Epoch 7/40
169/782 [=====>........................] - ETA: 4:39 - loss: 0.7192 - acc: 0.7429

782/782 [==============================] - 374s 479ms/step - loss: 0.7204 - acc: 0.7479 - val_loss: 1.0500 - val_acc: 0.6951

Epoch 00007: saving model to CIFAR10-imgaug-weights-improvement-07-0.70.hdf5
Epoch 8/40
169/782 [=====>........................] - ETA: 4:37 - loss: 0.6861 - acc: 0.7581

782/782 [==============================] - 373s 477ms/step - loss: 0.6772 - acc: 0.7642 - val_loss: 0.8302 - val_acc: 0.7441

Epoch 00008: saving model to CIFAR10-imgaug-weights-improvement-08-0.74.hdf5
Epoch 9/40
169/782 [=====>........................] - ETA: 4:38 - loss: 0.6581 - acc: 0.7726

232/782 [=======>......................] - ETA: 4:10 - loss: 0.6583 - acc: 0.7711

KeyboardInterrupt: ignored

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [15]:
# Save the trained weights in to .h5 format
model.save_weights("CIFAR10-imgaug_intermediate.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('CIFAR10-imgaug_intermediate.h5')

In [0]:
!ls